In [ ]:
import gdal
from os import listdir, makedirs
from os.path import isdir, join, isfile
import pandas as pd

In [ ]:
FOLDER_PATH = (
    "C:\\Users\\mracic\\Univerza v Ljubljani\\Fetai, Bujar - satimagets-master\\"
)
dfs = pd.read_excel(FOLDER_PATH + "Harmonisation_PS_S2.xlsx")

In [ ]:
locations = ["Radenci", "Izola", "Jesenice", "Kranj"]
INPUT_PATH = "Y:\\S2_10m\\"
OUTPUT_PATH = "Y:\\Harmonisation_PS_S2\\M3sat_S2output\\"

In [ ]:
sub_dfs.iloc[0]["PS_Path"]

In [ ]:
for loc_name in locations:
    print(loc_name)
    location_path = OUTPUT_PATH + loc_name + "5\\"
    sub_dfs = dfs[dfs["Location"] == loc_name]
    # Open the image of the selected area
    PS_PATH = sub_dfs.iloc[0]["PS_Path"]
    if isfile(PS_PATH):
        PS = gdal.Open(PS_PATH)
    else:
        print("Can't find file in path.")

    # Get bbox of AOI
    geoTransform = PS.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1] * PS.RasterXSize
    miny = maxy + geoTransform[5] * PS.RasterYSize
    data = None
    extent = [minx, miny, maxx, maxy]

    for year in range(2017, 2021):
        list_S2 = [
            o
            for o in listdir(INPUT_PATH + str(year))
            if isdir(join(INPUT_PATH + str(year), o))
        ]

        for observation in list_S2:
            # check if image is in dataset
            if sub_dfs["S2_filename"].str.contains(observation).any():
                # print(observation)
                tiffs = [
                    o
                    for o in listdir(INPUT_PATH + str(year) + "\\" + observation)
                    if ".tif" in o
                ]
                data = [o for o in tiffs if "p2atm_d96tm" in o]
                mask_data = [o for o in tiffs if "p2atm_mask_d96tm" in o]
                # crop the raster data
                gdal.Warp(
                    location_path + "\\" + str(year) + "\\" + observation + "_S2.tif",
                    INPUT_PATH + "\\" + str(year) + "\\" + observation + "\\" + data[0],
                    outputBounds=extent,
                    dstSRS="EPSG:32633",
                )
                gdal.Warp(
                    location_path
                    + "\\"
                    + str(year)
                    + "\\"
                    + observation
                    + "_mask_S2.tif",
                    INPUT_PATH
                    + "\\"
                    + str(year)
                    + "\\"
                    + observation
                    + "\\"
                    + mask_data[0],
                    outputBounds=extent,
                    dstSRS="EPSG:32633",
                )  #  srcSRS="EPSG:3794",

In [ ]:
# check for number of unique files
dfs[(dfs["Location"] == "Radenci") & (dfs["Year"] == 2020)][
    "S2_filename"
].unique().shape